# Read and Update Analyzers in Your Resource

This notebook demo how to read an existing analyzer and update it with a changed version.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/analyzer_management", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [ ]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"Analyzer details: {json.dumps(response['value'], indent=2)}")

## Get analyzer details with id

Remember the analyzer id when you create it. You can use the id to look up detail analyzer definitions afterwards.

In [ ]:
ANALYZER_ID = "PostCallAnalytics"

result = client.get_analyzer_detail_by_id(ANALYZER_ID)
print(json.dumps(result, indent=2))

## Write analyzer details into json file for editing
Write the analyzer config into a json file.

In [ ]:
output_dir = Path("../CU-Demo-Assets/analyzers/downloaded")

output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / f"{ANALYZER_ID}.json"
with open(output_file, "w") as f:
    json.dump(result, f, indent=2)
f.close()
print(f"Analyzer details written to {output_file}")



## Pause until you have made the updates in the downloaded analyzer json file

In [ ]:
pause = input("Press Enter to continue...")

## Update analyzer with id

Update an analyzer from a template.

In [ ]:
import uuid

ANALYZER_TEMPLATE = f"../CU-Demo-Assets/analyzers/downloaded/{ANALYZER_ID}.json"

client.delete_analyzer(ANALYZER_ID)
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE)
result = client.poll_result(response)

print(json.dumps(result, indent=2))